In [28]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [29]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
lemmatizer = WordNetLemmatizer()

words = []
labels = []
docs_x = []
docs_y = []

In [31]:
with open("faq.json",encoding="utf8") as faq:
    data=json.load(faq)

In [32]:
for intent in data["intents"]:
    for question in intent["question"]:
        temp=nltk.word_tokenize(question)
        words.extend(temp)

        docs_x.append(question)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [33]:
try:
    words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
except Exception as e:
    print(e)
words=sorted(list(set(words)))

labels=sorted(set(labels))

In [34]:
training = []
out_empty = [0]* len(labels)

In [35]:
docs_x

['hi',
 'hii',
 'hello',
 'hey',
 'hiii',
 'greetings',
 'hiiii',
 'hiiiii',
 'hellooooo',
 'hellooo',
 'hiiiiii',
 'hy',
 'gi',
 "hi'",
 'aurabot',
 'what is aurabot',
 'hi hi',
 'hello hello',
 'hello hello hello',
 'how are you',
 'how are u',
 'how r you',
 'how r u',
 'how are you doing',
 'how r u doing',
 'how are you?',
 'are you good',
 "i'm good",
 'i am good',
 'good',
 'fine',
 "i'm doing great",
 'great',
 'i am doing great',
 "i'm fine",
 'i am fine',
 'i m good',
 'i m doing great',
 'bye',
 'goodbye',
 'byee',
 'byeee',
 'see u later',
 'see you soon',
 'c u later',
 'c u soon',
 'see you latr',
 'see u later',
 'c you later',
 'see u soon',
 'c you soon',
 'by',
 'byeeeeee',
 'bye...',
 'bye bye',
 'who are you',
 'what is ur name',
 'what is your name',
 'whats your name',
 'who r you',
 'who are u',
 'who r u',
 'tell me about yourself',
 'tell me abt yourself',
 'tell me abt urself',
 'tell me about urself',
 'about yourself',
 'abt yourself',
 'about urself',
 'abt

In [36]:
for idx, doc in enumerate(docs_x):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)

    output_row = list(out_empty)
    output_row[labels.index(docs_y[idx])] = 1

    training.append([bow, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [37]:
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)

In [38]:
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               242176    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1240)              80600     
                                                                 
Total params: 331,032
Trainable params: 331,032
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
94/94 [==========================